In [1]:
import os
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Activation, Dropout, BatchNormalization

!pip install wandb
import wandb
from wandb.keras import WandbCallback
!pip install wandb --upgrade

  Attempting uninstall: wandb
    Found existing installation: wandb 0.12.12
    Uninstalling wandb-0.12.12:
      Successfully uninstalled wandb-0.12.12


In [2]:
#no_of_filters = [8, 16, 16, 32, 32]
kernel_size =   [3 , 5 , 5 , 5 , 5]
activation  =   tf.nn.relu
#dense_layer = 100

image_size = 200

batch_size = 50
epochs = 10
dropout=0.0
batch_norm=False
augment_data=False

In [3]:
#TensorFlow - Building the Model
def createCNN(image_size, no_of_filters, kernel_size, activation, batch_norm, dropout, dense_size):

    modeltf = Sequential()

    # layer1
    modeltf.add(Conv2D(input_shape=(image_size, image_size, 3), filters=no_of_filters[0], kernel_size=kernel_size[0], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer2
    modeltf.add(Conv2D(no_of_filters[1], kernel_size=kernel_size[1], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer3
    modeltf.add(Conv2D(no_of_filters[2], kernel_size=kernel_size[2], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer4
    modeltf.add(Conv2D(no_of_filters[3], kernel_size=kernel_size[3], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer5
    modeltf.add(Conv2D(no_of_filters[4], kernel_size=kernel_size[4], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    modeltf.add(Flatten())

    # dense layer
    modeltf.add(Dense(dense_layer, activation=activation))
    modeltf.add(Dropout(dropout))

    # output layer
    modeltf.add(Dense(10, activation=tf.nn.softmax))
    
    return modeltf

In [4]:
# Preparing dataset for training
def prepare_dataset(DATA_DIR="inaturalist_12K", augment_data=False):
    train_dir = os.path.join(DATA_DIR, "train")
    test_dir = os.path.join(DATA_DIR, "val")

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [5]:
#Code to set run name
def setRunName(no_of_filters = [8, 16, 16, 32, 32], augment_data=False, dropout=0.0, batch_norm=False):
    
    augment_data_options = {True: "Y", False: "N"}
    batch_norm_options = {True: "Y", False: "N"}

    run_name = "_".join(["filters", str(no_of_filters), "aug", augment_data_options[augment_data],
                      "drop", str(dropout), "norm", batch_norm_options[batch_norm]])
    
    return run_name;

In [7]:
#WandB integration for training and validation
def train():

    config_defaults = {
        "no_of_filters_1": 8,
        "no_of_filters_2": 16,
        "no_of_filters_3": 16,
        "no_of_filters_4": 32,
        "no_of_filters_5": 32,
        "augment_data": False,
        "dropout": 0.0,
        "batch_norm": False,
        "epochs": 10,
        "dense_size": 1000,
        "lr": 0.001
    }
    
    os.environ['WANDB_NOTEBOOK_NAME'] = 'A2_P1_2.ipynb'

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config
    wandb.run.name = setRunName([config.no_of_filters_1, config.no_of_filters_2, config.no_of_filters_3, config.no_of_filters_4, config.no_of_filters_5], config.augment_data, config.dropout, config.batch_norm)

    train_generator, val_generator, test_generator = prepare_dataset(augment_data=config.augment_data)
    model = createCNN(image_size, [config.no_of_filters_1, config.no_of_filters_2, config.no_of_filters_3, config.no_of_filters_4, config.no_of_filters_5], kernel_size, activation, config.batch_norm, config.dropout, config.dense_size)
    model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="accuracy")
    model.fit(train_generator, epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])

In [8]:
#Set up a sweep config
sweep_config = {
    "description": "Training and Checking the performance of CNN on validation data",
    "metric": {
    'name': 'val_accuracy',
    'goal': 'maximize'   
    },
    "method": "random",
    "project": "DL_CS6910_Assignment2",
    "parameters": {
        "no_of_filters_1": {
            "values": [4, 8]
        },
        "no_of_filters_2": {
            "values": [8, 16]
        },
        "no_of_filters_3": {
            "values": [16, 24]
        },
        "no_of_filters_4": {
            "values": [24, 32]
        },
        "no_of_filters_5": {
            "values": [32, 64]
        },
        "augment_data": {
            "values": [False, True]
        },
        "dropout": {
            "values": [0.0, 0.1, 0.2, 0.3]
        },
        "batch_norm": {
            "values": [False, True]
        },
        "epochs": {
            "values": [25]
        },
        "dense_size": {
            "values": [64, 100, 128]
        },
        "lr": {
            "values": [0.001, 0.01, 0.005]
        }
    }
}

# creating the sweep
sweep_id = wandb.sweep(sweep_config, project="DL_CS6910_Assignment2")

Create sweep with ID: 61cgxeqz
Sweep URL: https://wandb.ai/cs21m029_keyur_raval/DL_CS6910_Assignment2/sweeps/61cgxeqz


In [ ]:
wandb.agent(sweep_id, function=train, count = 100)

wandb: Agent Starting Run: j9kt0cg9 with config:
wandb: 	augment_data: False
wandb: 	batch_norm: True
wandb: 	dense_size: 100
wandb: 	dropout: 0
wandb: 	epochs: 25
wandb: 	lr: 0.005
wandb: 	no_of_filters_1: 8
wandb: 	no_of_filters_2: 8
wandb: 	no_of_filters_3: 16
wandb: 	no_of_filters_4: 32
wandb: 	no_of_filters_5: 64
wandb: Currently logged in as: cs21m029_keyur_raval (use `wandb login --relogin` to force relogin)


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/25
180/180 [==============================] - 77s 393ms/step - loss: 2.5937 - accuracy: 0.1621 - val_loss: 2.7130 - val_accuracy: 0.1331
Epoch 2/25
180/180 [==============================] - 68s 376ms/step - loss: 2.1778 - accuracy: 0.2000 - val_loss: 2.2550 - val_accuracy: 0.1502
Epoch 3/25
180/180 [==============================] - 69s 386ms/step - loss: 2.1477 - accuracy: 0.2068 - val_loss: 2.4065 - val_accuracy: 0.1692
Epoch 4/25
180/180 [==============================] - 69s 382ms/step - loss: 2.1366 - accuracy: 0.2178 - val_loss: 2.2841 - val_accuracy: 0.1752
Epoch 5/25
180/180 [==============================] - 68s 379ms/step - loss: 2.1144 - accuracy: 0.2249 - val_loss: 2.1577 - val_accuracy: 0.2022
Epoch 6/25
180/180 [==============================] - 68s 380ms/step - loss: 2.0979 - accuracy: 0.2273 - val_loss: 2.1977 - val_accuracy: 0.1862
Ep

accuracy,▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▇▇█
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▆▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▂▂▂▁
val_accuracy,▁▂▂▃▄▃▄▅▅▂▃▄▅▅▆█▆▇▇▆██▇▇▇
val_loss,█▃▅▄▂▃▂▂▂▆▄▃▂▂▁▁▂▁▁▃▁▂▅▆▇
accuracy,0.52211
best_epoch,17
best_val_loss,2.03601
epoch,24
loss,1.34846
val_accuracy,0.28529


wandb: Agent Starting Run: mmsdn308 with config:
wandb: 	augment_data: True
wandb: 	batch_norm: False
wandb: 	dense_size: 64
wandb: 	dropout: 0
wandb: 	epochs: 25
wandb: 	lr: 0.001
wandb: 	no_of_filters_1: 4
wandb: 	no_of_filters_2: 16
wandb: 	no_of_filters_3: 16
wandb: 	no_of_filters_4: 24
wandb: 	no_of_filters_5: 32


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/25
180/180 [==============================] - 131s 710ms/step - loss: 2.2875 - accuracy: 0.1154 - val_loss: 2.2346 - val_accuracy: 0.1922
Epoch 2/25
180/180 [==============================] - 123s 685ms/step - loss: 2.2067 - accuracy: 0.1832 - val_loss: 2.1970 - val_accuracy: 0.1992
Epoch 3/25
180/180 [==============================] - 117s 649ms/step - loss: 2.1539 - accuracy: 0.2144 - val_loss: 2.1483 - val_accuracy: 0.2092
Epoch 4/25
180/180 [==============================] - 121s 671ms/step - loss: 2.1350 - accuracy: 0.2218 - val_loss: 2.1407 - val_accuracy: 0.2292
Epoch 5/25
180/180 [==============================] - 119s 658ms/step - loss: 2.1039 - accuracy: 0.2401 - val_loss: 2.1215 - val_accuracy: 0.2182
Epoch 6/25
180/180 [==============================] - 117s 648ms/step - loss: 2.0824 - accuracy: 0.2477 - val_loss: 2.1026 - val_accuracy: 0.2

...

wandb: Agent Starting Run: o15pxp18 with config:
wandb: 	augment_data: True
wandb: 	batch_norm: True
wandb: 	dense_size: 128
wandb: 	dropout: 0.2
wandb: 	epochs: 25
wandb: 	lr: 0.005
wandb: 	no_of_filters_1: 8
wandb: 	no_of_filters_2: 16
wandb: 	no_of_filters_3: 24
wandb: 	no_of_filters_4: 32
wandb: 	no_of_filters_5: 32


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/25
180/180 [==============================] - 121s 670ms/step - loss: 2.4397 - accuracy: 0.1624 - val_loss: 2.6932 - val_accuracy: 0.1141
Epoch 2/25
180/180 [==============================] - 120s 667ms/step - loss: 2.1938 - accuracy: 0.1969 - val_loss: 2.1658 - val_accuracy: 0.2182
Epoch 3/25
180/180 [==============================] - 124s 687ms/step - loss: 2.1722 - accuracy: 0.2099 - val_loss: 2.1350 - val_accuracy: 0.2382
Epoch 4/25
180/180 [==============================] - 124s 686ms/step - loss: 2.1474 - accuracy: 0.2138 - val_loss: 2.1190 - val_accuracy: 0.2232
Epoch 5/25
180/180 [==============================] - 126s 698ms/step - loss: 2.1335 - accuracy: 0.2201 - val_loss: 2.1557 - val_accuracy: 0.2342
Epoch 6/25
180/180 [==============================] - 125s 692ms/step - loss: 2.1159 - accuracy: 0.2256 - val_loss: 2.1752 - val_accuracy: 0.2

accuracy,▁▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▄▅▄▅▄▅▅▅▄▇▆▆▆▆▇▇▅▆▆▇█▇▇█
val_loss,█▃▃▃▃▃▃▂▃▃▂▂▃▃▂▁▂▂▂▂▁▁▁▂▁
accuracy,0.31411
best_epoch,21
best_val_loss,1.91816
epoch,24
loss,1.91016
val_accuracy,0.33534


wandb: Agent Starting Run: 3cs7fkmj with config:
wandb: 	augment_data: False
wandb: 	batch_norm: True
wandb: 	dense_size: 100
wandb: 	dropout: 0.3
wandb: 	epochs: 25
wandb: 	lr: 0.005
wandb: 	no_of_filters_1: 8
wandb: 	no_of_filters_2: 16
wandb: 	no_of_filters_3: 16
wandb: 	no_of_filters_4: 32
wandb: 	no_of_filters_5: 64


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/25
180/180 [==============================] - 72s 394ms/step - loss: 2.4952 - accuracy: 0.1233 - val_loss: 2.2963 - val_accuracy: 0.1181
Epoch 2/25
180/180 [==============================] - 70s 388ms/step - loss: 2.2855 - accuracy: 0.1293 - val_loss: 2.2630 - val_accuracy: 0.1552
Epoch 3/25
180/180 [==============================] - 73s 403ms/step - loss: 2.2607 - accuracy: 0.1501 - val_loss: 2.2800 - val_accuracy: 0.1251
Epoch 4/25
180/180 [==============================] - 71s 393ms/step - loss: 2.2482 - accuracy: 0.1570 - val_loss: 2.3046 - val_accuracy: 0.1782
Epoch 5/25
180/180 [==============================] - 73s 404ms/step - loss: 2.2285 - accuracy: 0.1691 - val_loss: 2.1992 - val_accuracy: 0.1752
Epoch 6/25
180/180 [==============================] - 70s 387ms/step - loss: 2.2091 - accuracy: 0.1773 - val_loss: 2.1903 - val_accuracy: 0.1722
Ep